In [1]:

catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
latitude = 41.41505315463394
longitude = -122.19468933831948
buffer = 0.1
bbox = [longitude - buffer, latitude - buffer, longitude + buffer, latitude + buffer]
search = catalog.search(collections="3dep-seamless", bbox=bbox)
items = list(search.get_items())
print(f"{len(items)} items found")

items_high_res = [
    item
    for item in items
    if item.properties["gsd"] == 10
]
print(f"{len(items_high_res)} high res items found")



2 items found
1 high res items found


In [2]:
from odc import stac
import yaml

configuration_str = """---
3dep-seamless:
  measurements:
    '*':
      dtype: float32
      nodata: -999999
      units: 'm'
"""
configuration = yaml.load(configuration_str, Loader=yaml.CSafeLoader)
datasets = stac.stac2ds(items_high_res, configuration)

In [6]:
import planetary_computer
from datacube.model import Dataset
from odc.algo import io

@planetary_computer.sign.register(Dataset)
def sign_odc_dataset(dataset: Dataset) -> Dataset:
    for m in dataset.metadata_doc["measurements"].values():
        m["path"] = planetary_computer.sign(m["path"])
    return dataset

signed_datasets = [planetary_computer.sign(dataset) for dataset in datasets]
data = io.dc_load(signed_datasets)

TypeError: __new__() takes 7 positional arguments but 10 were given